# Génie Logiciel
## TD2 - MySQL et Python 

Source des tutoriels :

* W3 Schools : https://www.w3schools.com/python/python_mysql_getstarted.asp
* RealPython : https://realpython.com/python-mysql/#creating-altering-and-dropping-a-table

### Driver

Il est possible de se connecter avec tout type de base de données avec presque tous les langages de programmation. Pour cela, ces langages ont besoin d'un driver ou connecteur leur permettant d'accéder et manipuler les bases de données. En Python par exemple, les drivers suivants sont nécessaires pour se connecter aux différentes variantes de SQL :
* SQLite : sqlite3
* PostgreSQL : psycopg
* MySQL : mysql-connector

Pour ce cours, nous installerons donc mysql-connector

In [1]:
!pip install mysql-connector-python

     |████████████████████████████████| 581 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 486 kB 1.3 MB/s eta 0:00:01


In [1]:
# pour tester si l'installation est réussie
import mysql.connector


## Se connecter à un serveur MySQL

Pour pouvoir accéder et manipuler les bases de données, il est d'abord nécessaire de se connecter au serveur MySQL qui les contient. Il faut donc connaître l'adresse IP, l'identifiant et le mot de passe. Si votre serveur tourne en local (donc sur votre machine), le serveur sera "localhost".

Note : ci-dessous, on donne les identifiants en dur. C'est une très mauvaise pratique, et peut engendrer des failles de sécurité. Si les identifiants doivent être contenus près de votre code, enregistrez les dans un fichier sécurisé. 


In [2]:
from getpass import getpass
from mysql.connector import connect, Error

user_input = 'root'
# remplacez par votre mot de passe
password = '******'

try:
    # on obtient une variable connection de type MySQLConnection avec laquelle on peut intéragir avec le serveur
    connection = connect(
        host="localhost",
        user = user_input,
        password = password
        # user=input("Enter username: "),
        # password=getpass("Enter password: ")
    )
    print(connection)
# gestion de toutes erreurs de connection
except Error as e:
    print(e)


## Effectuer des requêtes

Pour effectuer des requêtes, on a besoin d'un objet de type **cursor**

In [5]:
# requête pour voir les bases existantes dans le serveur
# remarquez que la requête ne termine pas par ; . Le cursor se charge de l'ajouter
query_show = "SHOW DATABASES"

with connection.cursor() as cursor:

    cursor.execute(query_show) 
    # pour récupérer les résultats obtenus par la requête, on doit appeler la méthode fetchall()
    for x in cursor.fetchall():
        print(x)

('CentreTesniere',)
('CT',)
('CT3',)
('France',)
('information_schema',)
('mydatabase',)
('mysql',)
('new_schema',)
('performance_schema',)
('sys',)


In [4]:
with connection.cursor() as cursor:

    cursor.execute("CREATE DATABASE mydatabase")


## Connection à une base de données

De la même manière, on peut se connecter à une base à l'aide du cursor

In [6]:
dbname = "mydatabase"
query_connect_db = f"USE {dbname}"

with connection.cursor() as cursor:

    cursor.execute(query_connect_db)


In [41]:
# on peut également se connecter à la base en même temps que l'on se connecte au serveur
try:
    # on obtient une variable connection de type MySQLConnection avec laquelle on peut intéragir avec le serveur
    connection = connect(
        host="localhost",
        user = user_input,
        password = password,
        database=dbname
        # user=input("Enter username: "),
        # password=getpass("Enter password: ")
    )
    print(connection)
# gestion de toutes erreurs de connection
except Error as e:
    print(e)

## Création d'une table et injection de données

In [7]:
create_movies_table_query = """
CREATE TABLE movies(
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    title VARCHAR(100),
    release_year YEAR(4),
    genre VARCHAR(100),
    collection_in_mil INT
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_movies_table_query)
    # on note ici que l'on fait appel à la méthode commit(), qui est nécessaire lorsque l'on modifie la base
    connection.commit()


In [9]:
create_reviewers_table_query = """
CREATE TABLE reviewers (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    first_name VARCHAR(100),
    last_name VARCHAR(100)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_reviewers_table_query)
    connection.commit()


In [10]:
create_ratings_table_query = """
CREATE TABLE ratings (
    movie_id INT,
    reviewer_id INT,
    rating DECIMAL(2,1),
    FOREIGN KEY(movie_id) REFERENCES movies(id),
    FOREIGN KEY(reviewer_id) REFERENCES reviewers(id),
    PRIMARY KEY(movie_id, reviewer_id)
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_ratings_table_query)
    connection.commit()


## Description d'une table

In [11]:
show_table_query = "DESCRIBE movies"
with connection.cursor() as cursor:
    cursor.execute(show_table_query)
    for x in cursor.fetchall():
        print(x)

('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(100)', 'YES', '', None, '')
('release_year', 'year', 'YES', '', None, '')
('genre', 'varchar(100)', 'YES', '', None, '')
('collection_in_mil', 'int', 'YES', '', None, '')


## Insérer des données

Suivant le nombre de données, il y a deux méthodes pour les injecter: execute() et executemany()

In [13]:
# avec execute()
insert_movies_query = """
INSERT INTO movies (title, release_year, genre, collection_in_mil)
VALUES
    ("Forrest Gump", 1994, "Drama", 330.2),
    ("3 Idiots", 2009, "Drama", 2.4),
    ("Eternal Sunshine of the Spotless Mind", 2004, "Drama", 34.5),
    ("Good Will Hunting", 1997, "Drama", 138.1),
    ("Skyfall", 2012, "Action", 304.6),
    ("Gladiator", 2000, "Action", 188.7),
    ("Black", 2005, "Drama", 3.0),
    ("Titanic", 1997, "Romance", 659.2),
    ("The Shawshank Redemption", 1994, "Drama",28.4),
    ("Udaan", 2010, "Drama", 1.5),
    ("Home Alone", 1990, "Comedy", 286.9),
    ("Casablanca", 1942, "Romance", 1.0),
    ("Avengers: Endgame", 2019, "Action", 858.8),
    ("Night of the Living Dead", 1968, "Horror", 2.5),
    ("The Godfather", 1972, "Crime", 135.6),
    ("Haider", 2014, "Action", 4.2),
    ("Inception", 2010, "Adventure", 293.7),
    ("Evil", 2003, "Horror", 1.3),
    ("Toy Story 4", 2019, "Animation", 434.9),
    ("Air Force One", 1997, "Drama", 138.1),
    ("The Dark Knight", 2008, "Action",535.4),
    ("Bhaag Milkha Bhaag", 2013, "Sport", 4.1),
    ("The Lion King", 1994, "Animation", 423.6),
    ("Pulp Fiction", 1994, "Crime", 108.8),
    ("Kai Po Che", 2013, "Sport", 6.0),
    ("Beasts of No Nation", 2015, "War", 1.4),
    ("Andadhun", 2018, "Thriller", 2.9),
    ("The Silence of the Lambs", 1991, "Crime", 68.2),
    ("Deadpool", 2016, "Action", 363.6),
    ("Drishyam", 2015, "Mystery", 3.0)
"""
with connection.cursor() as cursor:
    cursor.execute(insert_movies_query)
    # ne pas oublier d'appeler commit()
    connection.commit()


In [14]:
# avec executemany(). Notez les %s dans VALUES, ainsi que la liste de tuples reviewers_records
insert_reviewers_query = """
INSERT INTO reviewers
(first_name, last_name)
VALUES ( %s, %s )
"""
reviewers_records = [
    ("Chaitanya", "Baweja"),
    ("Mary", "Cooper"),
    ("John", "Wayne"),
    ("Thomas", "Stoneman"),
    ("Penny", "Hofstadter"),
    ("Mitchell", "Marsh"),
    ("Wyatt", "Skaggs"),
    ("Andre", "Veiga"),
    ("Sheldon", "Cooper"),
    ("Kimbra", "Masters"),
    ("Kat", "Dennings"),
    ("Bruce", "Wayne"),
    ("Domingo", "Cortes"),
    ("Rajesh", "Koothrappali"),
    ("Ben", "Glocker"),
    ("Mahinder", "Dhoni"),
    ("Akbar", "Khan"),
    ("Howard", "Wolowitz"),
    ("Pinkie", "Petit"),
    ("Gurkaran", "Singh"),
    ("Amy", "Farah Fowler"),
    ("Marlon", "Crafford"),
]
with connection.cursor() as cursor:
    cursor.executemany(insert_reviewers_query, reviewers_records)
    connection.commit()


In [15]:
insert_ratings_query = """
INSERT INTO ratings
(rating, movie_id, reviewer_id)
VALUES ( %s, %s, %s)
"""
ratings_records = [
    (6.4, 17, 5), (5.6, 19, 1), (6.3, 22, 14), (5.1, 21, 17),
    (5.0, 5, 5), (6.5, 21, 5), (8.5, 30, 13), (9.7, 6, 4),
    (8.5, 24, 12), (9.9, 14, 9), (8.7, 26, 14), (9.9, 6, 10),
    (5.1, 30, 6), (5.4, 18, 16), (6.2, 6, 20), (7.3, 21, 19),
    (8.1, 17, 18), (5.0, 7, 2), (9.8, 23, 3), (8.0, 22, 9),
    (8.5, 11, 13), (5.0, 5, 11), (5.7, 8, 2), (7.6, 25, 19),
    (5.2, 18, 15), (9.7, 13, 3), (5.8, 18, 8), (5.8, 30, 15),
    (8.4, 21, 18), (6.2, 23, 16), (7.0, 10, 18), (9.5, 30, 20),
    (8.9, 3, 19), (6.4, 12, 2), (7.8, 12, 22), (9.9, 15, 13),
    (7.5, 20, 17), (9.0, 25, 6), (8.5, 23, 2), (5.3, 30, 17),
    (6.4, 5, 10), (8.1, 5, 21), (5.7, 22, 1), (6.3, 28, 4),
    (9.8, 13, 1)
]
with connection.cursor() as cursor:
    cursor.executemany(insert_ratings_query, ratings_records)
    connection.commit()


## Sélectionner des données

In [16]:
select_movies_query = "SELECT title, release_year FROM movies LIMIT 5"

with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for row in cursor.fetchall():
        print(row)

('Forrest Gump', 1994)
('3 Idiots', 2009)
('Eternal Sunshine of the Spotless Mind', 2004)
('Good Will Hunting', 1997)
('Skyfall', 2012)


In [17]:
select_movies_query = """
    SELECT title, collection_in_mil
    FROM movies
    WHERE collection_in_mil > 300
    ORDER BY collection_in_mil DESC
    """

with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

('Avengers: Endgame', 859)
('Titanic', 659)
('The Dark Knight', 535)
('Toy Story 4', 435)
('The Lion King', 424)
('Deadpool', 364)
('Forrest Gump', 330)
('Skyfall', 305)


In [42]:
# la méthode fetchmany() permet de spécifier le nombre de résultat à retourner
select_movies_query = """
    SELECT title, collection_in_mil
    FROM movies
    """

with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchmany(size=15):
        print(movie)
    # important d'apeler fetchall() afin de vider le cursor et de permettre de nouvelles requêtes
    # sinon vous aurez une erreur InternalError: Unread result
    cursor.fetchall()

('Forrest Gump', 330)
('3 Idiots', 2)
('Eternal Sunshine of the Spotless Mind', 35)
('Good Will Hunting', 138)
('Skyfall', 305)
('Gladiator', 189)
('Black', 3)
('Titanic', 659)
('The Shawshank Redemption', 28)
('Udaan', 2)
('Home Alone', 287)
('Casablanca', 1)
('Avengers: Endgame', 859)
('Night of the Living Dead', 3)
('The Godfather', 136)


## Jointure

In [18]:
select_movies_query = """
SELECT title, AVG(rating) as average_rating
FROM ratings
INNER JOIN movies
    ON movies.id = ratings.movie_id
GROUP BY movie_id
ORDER BY average_rating DESC
LIMIT 5
"""
with connection.cursor() as cursor:
    cursor.execute(select_movies_query)
    for movie in cursor.fetchall():
        print(movie)

('The Godfather', Decimal('9.90000'))
('Night of the Living Dead', Decimal('9.90000'))
('Avengers: Endgame', Decimal('9.75000'))
('Eternal Sunshine of the Spotless Mind', Decimal('8.90000'))
('Beasts of No Nation', Decimal('8.70000'))


# Object Relational Mapping (ORM)

Il existe d'autres types interfaces dites ORM qui permettent de manipuler des bases de données avec Python de façon orientée objet. Un autre avantage de ces interfaces est qu'elles permettent de manipuler différents types de bases de manière similaire (SQL, PostgreSQL...). Les principales sont :

* SQLAchemy
* peewee
* Django ORM (que l'on verra plus tard)

------------

## Cas pratique

En manipulant vos bases de données uniquement via Python, réalisez les tâches suivantes :
* Créez une base "CT3"

* Créez les tables suivantes : "Cours", "Enseignants", "EnseignantsEtudiants", "Etudiants", "InfosEnseignants". Basez vous sur les fichiers CSV contenus dans le dossier data pour nommer les colonnes et accorder les bons types

* Ecrivez un code qui va parcourir chaque fichier CSV du dossier data et intègre ses données à la table correspondante.

* Faites des requêtes suivantes :
    * Affichez le nom, le prenom et la ville de tous les enseignants
    * Compter les occurrences de chaque ville
    * Voir les enseignants et les étudiants qui ont cours avec


* Réalisez les actions suivantes :
    * Modifiez la valeur "nom" pour "Dupont" pour l'élément ayant l'identifiant 1
    * Modifiez la valeur de "ville" pour "Saint-Claude" et "codePostal" pour "39000" pour l'élément ayant l'identifiant 3
    * Supprimez tous les éléments dont le code postal est "25000"

Vous pouvez bien entendu accéder à MySQL Workbench pour contrôler que vos actions se réalisent bien, mais pas pour réaliser les étapes ci-dessus.

## Solution

In [4]:
from getpass import getpass
from mysql.connector import connect, Error

user_input = 'root'
# remplacez par votre mot de passe
password = "*******"

try:
    # on obtient une variable connection de type MySQLConnection avec laquelle on peut intéragir avec le serveur
    connection = connect(
        host="localhost",
        user = user_input,
        password = password,

        # user=input("Enter username: "),
        # password=getpass("Enter password: ")
    )
    print(connection)
# gestion de toutes erreurs de connection
except Error as e:
    print(e)


In [5]:
# creation de la base CT3
dbname = 'CT3'
with connection.cursor() as cursor:

    cursor.execute(f"CREATE DATABASE {dbname}")


In [8]:
# utilisation de la base
with connection.cursor() as cursor:

    cursor.execute(f"USE {dbname}")

In [13]:
# création des tables. Toutes les requêtes sont contenues dans une liste. 
# on procède à chaque requête à l'aide d'une boucle
create_tables_queries = [
"""
CREATE TABLE Enseignants (
    enseignantID INT AUTO_INCREMENT PRIMARY KEY,
    nom VARCHAR(100),
    prenom VARCHAR(100)
)
""",  
"""
CREATE TABLE Etudiants (
    etudiantID INT AUTO_INCREMENT PRIMARY KEY,
    nom VARCHAR(100),
    prenom VARCHAR(100),
    age INT
)
""", 
"""
CREATE TABLE Cours (
    coursID INT AUTO_INCREMENT PRIMARY KEY,
    nom VARCHAR(100),
    salle VARCHAR(100),
    enseignantID INT,
    FOREIGN KEY(enseignantID) REFERENCES Enseignants(enseignantID)
)
""",
"""
CREATE TABLE InfosEnseignants (
    enseignantID INT,
    age INT,
    ville VARCHAR(100),
    codePostal INT,
    dateNaissance DATE,
    FOREIGN KEY(enseignantID) REFERENCES Enseignants(enseignantID)
)
""",
"""
CREATE TABLE EnseignantsEtudiants (
    id INT AUTO_INCREMENT PRIMARY KEY,
    enseignantID INT,
    etudiantID INT,
    FOREIGN KEY(enseignantID) REFERENCES Enseignants(enseignantID),
    FOREIGN KEY(etudiantID) REFERENCES Etudiants(etudiantID)

)
""",
]

for create_query in create_tables_queries:
    with connection.cursor() as cursor:
        cursor.execute(create_query)
        connection.commit()

In [41]:
import os 

def insertDataFromFile(filepath):
    print(filepath)
    # on va avoir besoin du nom de fichier pour ajouter les données à la bonne table
    filename = os.path.basename(filepath)
    filename = filename.replace('.csv', '')

    # on récupère les données
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.read()
        lines = lines.split('\n')

        header = lines[0]
        # on se débarasse de la première ligne, qui est le header
        lines = lines[1:]

    num_col = len(header.split(','))

    insert_query = f"""
        INSERT INTO {filename}
        ({header})
        VALUES ({','.join(['%s' for i in range(num_col)])})
    """
    print(insert_query)
    # on transforme chaque valeur en tuple pour pouvoir l'insérer dans la base de données
    insert_data = []
    for line in lines:
        data = line.split(',')
        insert_data.append(data)

    with connection.cursor() as cursor:
        cursor.executemany(insert_query, insert_data)
        connection.commit()
    # break

insertDataFromFile('data/Enseignants.csv')
insertDataFromFile('data/InfosEnseignants.csv')
insertDataFromFile('data/Etudiants.csv')
insertDataFromFile('data/EnseignantsEtudiants.csv')
insertDataFromFile('data/Cours.csv')



data/InfosEnseignants.csv

        INSERT INTO InfosEnseignants
        (enseignantID,age,ville,codePostal,dateNaissance)
        VALUES (%s,%s,%s,%s,%s)
    
data/Etudiants.csv

        INSERT INTO Etudiants
        (etudiantID,nom,prenom,age)
        VALUES (%s,%s,%s,%s)
    
data/EnseignantsEtudiants.csv

        INSERT INTO EnseignantsEtudiants
        (id,enseignantID,etudiantID)
        VALUES (%s,%s,%s)
    
data/Cours.csv

        INSERT INTO Cours
        (coursID,nom,salle,enseignantID)
        VALUES (%s,%s,%s,%s)
    


In [45]:

select_query = f'''
SELECT nom, prenom, ville FROM CT3.Enseignants
INNER JOIN CT3.InfosEnseignants ON CT3.Enseignants.enseignantID = CT3.InfosEnseignants.enseignantID
'''

# select_query =  '''SELECT COUNT(ville), ville FROM CT3.InfosEnseignants GROUP BY ville'''

# select_query = '''SELECT * FROM CT3.Enseignants
# INNER JOIN CT3.EnseignantsEtudiants ON CT3.enseignants.enseignantID = CT3.EnseignantsEtudiants.enseignantID
# INNER JOIN CT3.etudiants ON CT3.etudiants.etudiantID = CT3.EnseignantsEtudiants.etudiantID
# '''

with connection.cursor() as cursor:
    cursor.execute(select_query)
    for value in cursor.fetchall():
        print(value)

('"Gutehrle"', '"Nicolas"', '"Dijon"')
('"Guedat"', '"Nathan"', '"Besancon"')
('"Ozturk"', '"Yagmur"', '"Besancon"')
('"Nomblot"', '"Aurelie"', '"Besancon"')


In [50]:
update_query = '''
UPDATE CT3.Enseignants SET nom="Dupont" WHERE CT3.Enseignants.enseignantID=1'''

update_query = ''' 
UPDATE CT3.InfosEnseignants SET ville="Saint-Claude", codePostal=39000 WHERE CT3.InfosEnseignants.enseignantID=3;
'''

update_query = ''' 
DELETE FROM CT3.InfosEnseignants WHERE codePostal=25000
'''

with connection.cursor() as cursor:
    cursor.execute(update_query)
    connection.commit()
